Copyright **Jacob Martin and Paolo Raiteri**, January 2021

## Numerical solution of chemical equilibrium problems #4
Try now to solve the equilibrium problem for the equilibrium of the phosphate ion in a buffer solution where the pH is kept constant by external titration.
The relevant reactions with their equilibrium constants are listed below

\begin{eqnarray}
\mathrm{H_3PO_4^{0} \rightleftharpoons H_2PO_4^{-} + H^+} \qquad &;& \qquad pK_{1} = 2.168 \\
\mathrm{H_2PO_4^{-} \rightleftharpoons HPO_4^{2-} + H^+} \qquad &;& \qquad pK_{2} = 7.207 \\
\mathrm{HPO_4^{2-} \rightleftharpoons PO_4^{3-} + H^+} \qquad &;& \qquad pK_{3} = 12.346 \\
\end{eqnarray}


- Click `Download CSV` to export the data as a CSV file to verify your result.


In [ ]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

In [ ]:
def initialise():
    global nPoints
    global concA, concB, concC, concD, pH
    global pKeq1, pKeq2, pKeq3
    global delta

    nPoints = 50
    concA = 0.1
    concB = 0.1
    concC = 0.1
    concD = 0.1
    pH = 7
    pKeq1 = 2.168
    pKeq2 = 7.207
    pKeq3 = 12.346
    delta = 0.1

def addLine(t,x,y,z,l,q1,q2,q3,res):
    var_list = []
    var_list.append(t)
    var_list.append(x)
    var_list.append(y)
    var_list.append(z)
    var_list.append(l)
    var_list.append(q1)
    var_list.append(q2)
    var_list.append(q3)
    res.loc[len(res)] = var_list

initialise()


In [ ]:
respath = os.path.join(os.getcwd(), "..", "results.csv")

out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))

    
def force(Q,pk):
    if (Q > 0):
        force = - (math.log(Q,10) + pk)
    else:
        force = 1.

    return force

def calc1(btn):
    out_P.clear_output()
    
    if os.path.exists(respath):
        os.remove(respath)
    res = pd.DataFrame(columns=["step" 
                                , "H$_3$PO$_4^{0}$" 
                                , "H$_2$PO$_4^{-}$" 
                                , "HPO$_4^{2-}$" 
                                , "PO$_4^{3-}$"
                                , "$Q_1/K_1$"
                                , "$Q_2/K_2$"
                                , "$Q_3/K_3$"
                               ])

    A  = float(concA_text.value)
    B  = float(concB_text.value)
    C  = float(concC_text.value)
    D  = float(concD_text.value)
    H  = math.pow(10,-float(pH_text.value))

    dx = float(delta_text.value)
    pk1 = float(pKeq1_text.value)
    pk2 = float(pKeq2_text.value)
    pk3 = float(pKeq3_text.value)
    n  = int(nPoints_text.value)

    k1 = math.pow(10,-pk1)
    k2 = math.pow(10,-pk2)
    k3 = math.pow(10,-pk3)
    
    Q1 = B * H / A
    Q2 = C * H / B
    Q3 = D * H / C
    
    addLine(1,A,B,C,D,
            Q1/k1,
            Q2/k2,
            Q3/k3,
            res)
    
    for i in range(0, n):
        f1 = force(Q1,pk1)
        f2 = force(Q2,pk2)
        f3 = force(Q3,pk3)

        dc = min(A,B)
        dc = min(C,dc)
        dc = min(D,dc)
        if (dc == 0):
            dc = 0.0001

        dx1 = min(A,B)*(dx * f1)
        dx2 = min(B,C)*(dx * f2)
        dx3 = min(C,D)*(dx * f3)
        
        A = A - dx1
        B = B - dx2 + dx1
        C = C - dx3 + dx2
        D = D + dx3

        Q1 = B * H / A            
        Q2 = C * H / B
        Q3 = D * H / C

        addLine(i+1,A,B,C,D,
                Q1/k1,
                Q2/k2,
                Q3/k3,
                res)

    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(n))

btn_calc1 = ipw.Button(description="Get Data", layout=ipw.Layout(width="150px"))
btn_calc1.on_click(calc1)

rows = []

# Equilibrium constant
pKeq1_text = ipw.Text(str(pKeq1))
pKeq2_text = ipw.Text(str(pKeq2))
pKeq3_text = ipw.Text(str(pKeq3))

# Initial concentrations
concA_text = ipw.Text(str(concA))
concB_text = ipw.Text(str(concB))
concC_text = ipw.Text(str(concC))
concD_text = ipw.Text(str(concD))
pH_text = ipw.Text(str(pH))

# delta concentration
delta_text = ipw.Text(str(delta))

# Nmber of data points
nPoints_text = ipw.Text(str(nPoints))

rows.append(ipw.HBox([ipw.Label('Initial concentration of "H$_3$PO$_4^{0}$" :  '),concA_text]))
rows.append(ipw.HBox([ipw.Label('Initial concentration of "H$_2$PO$_4^{-}$" :  '),concB_text]))
rows.append(ipw.HBox([ipw.Label('Initial concentration of "HPO$_4^{2-}$"    :  '),concC_text]))
rows.append(ipw.HBox([ipw.Label('Initial concentration of "PO$_4^{3-}$"    :  '),concD_text]))
rows.append(ipw.HBox([ipw.Label('Buffer pH    :  '),pH_text]))
rows.append(ipw.HBox([ipw.Label('Equilibrium constant ($pK_1$)          :  '),pKeq1_text]))
rows.append(ipw.HBox([ipw.Label('Equilibrium constant ($pK_2$)          :  '),pKeq2_text]))
rows.append(ipw.HBox([ipw.Label('Equilibrium constant ($pK_3$)          :  '),pKeq3_text]))
rows.append(ipw.HBox([ipw.Label('Delta concentration                   :  '),delta_text]))
rows.append(ipw.HBox([ipw.Label('Number of data point required         :  '),nPoints_text]))

rows.append(ipw.HBox([btn_calc1]))

rows.append(ipw.HBox([out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)